In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets torch
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import json
import torch
from datasets import Dataset
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import pandas as pd
from transformers import pipeline
import math

In [4]:
# ✅ Define File Path
dataset_path = "/content/drive/MyDrive/DSGP/dataset/qa_dataset.json"

In [5]:
# ✅ Load JSON Data
with open(dataset_path, "r", encoding="utf-8") as f:
    dataset = json.load(f)

In [6]:

for i in range(5):  # Check first 5 examples
    print(f"🔹 Question: {dataset[i]['question']}")
    print(f"🔹 Retrieved Chunk: {dataset[i]['retrieved_chunk']}")
    print(f"🔹 Enhanced Response: {dataset[i]['enhanced_response']}\n")


🔹 Question: What is the scientific name of the Red-Vented Bulbul?
🔹 Retrieved Chunk: Pycnonotus cafer
🔹 Enhanced Response: The scientific name of the Red-Vented Bulbul is *Pycnonotus cafer*.

🔹 Question: What is the conservation status of the Red-Vented Bulbul?
🔹 Retrieved Chunk: Least Concern
🔹 Enhanced Response: The Red-Vented Bulbul is classified as 'Least Concern', meaning it is not currently endangered.

🔹 Question: What are the distinctive features of the Red-Vented Bulbul?
🔹 Retrieved Chunk: Black crest, white rump, red vent under tail
🔹 Enhanced Response: The Red-Vented Bulbul has the following distinctive features: Black crest, white rump, red vent under tail.

🔹 Question: What size is the Red-Vented Bulbul?
🔹 Retrieved Chunk: Medium-sized
🔹 Enhanced Response: The Red-Vented Bulbul is Medium-sized in size.

🔹 Question: Where does the Red-Vented Bulbul live?
🔹 Retrieved Chunk: Often seen high in trees or perched on wires in urban and rural areas; generally prefers scrubby edge 

In [7]:
dataset = pd.read_json(dataset_path)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   question           200 non-null    object
 1   retrieved_chunk    200 non-null    object
 2   enhanced_response  200 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB


In [8]:
# ✅ Convert to List of Text Pairs (For Training)
formatted_data = []
for index, row in dataset.iterrows():  # Iterate over rows using iterrows()
    question = row["question"]
    retrieved_chunk = row["retrieved_chunk"]
    enhanced_response = row["enhanced_response"]

    # GPT-2 Prompt Style
    text_input = f"Instruction: Enhance the following bird description with a smooth and engaging narrative.\nQuestion: {question}\nRetrieved Chunk: {retrieved_chunk}\nEnhanced Response: {enhanced_response}"

    formatted_data.append({"text": text_input})

In [9]:
dataset = Dataset.from_list(formatted_data)

In [10]:
print(dataset[:5])

{'text': ['Instruction: Enhance the following bird description with a smooth and engaging narrative.\nQuestion: What is the scientific name of the Red-Vented Bulbul?\nRetrieved Chunk: Pycnonotus cafer\nEnhanced Response: The scientific name of the Red-Vented Bulbul is *Pycnonotus cafer*.', "Instruction: Enhance the following bird description with a smooth and engaging narrative.\nQuestion: What is the conservation status of the Red-Vented Bulbul?\nRetrieved Chunk: Least Concern\nEnhanced Response: The Red-Vented Bulbul is classified as 'Least Concern', meaning it is not currently endangered.", 'Instruction: Enhance the following bird description with a smooth and engaging narrative.\nQuestion: What are the distinctive features of the Red-Vented Bulbul?\nRetrieved Chunk: Black crest, white rump, red vent under tail\nEnhanced Response: The Red-Vented Bulbul has the following distinctive features: Black crest, white rump, red vent under tail.', 'Instruction: Enhance the following bird des

In [11]:
# ✅ Load GPT-2 Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 does not have a pad token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [12]:
# ✅ Tokenize Dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

print("✅ Dataset Prepared for Training!")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

✅ Dataset Prepared for Training!


In [13]:
# ✅ Load GPT-2 Model
model = GPT2LMHeadModel.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
# ✅ Split dataset into 80% Train & 20% Evaluation
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)

# ✅ Assign Train & Eval Datasets
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

In [15]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DSGP/models/gpt2_finetuned2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,  # ✅ Increase batch size (better learning)
    per_device_eval_batch_size=4,
    num_train_epochs=7,  # ✅ Increase epochs (more exposure)
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/DSGP/logs",
    logging_steps=10,
    save_total_limit=2
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# ✅ Define Data Collator (Helps with padding)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [17]:
# ✅ Initialize Trainer with Eval Dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # ✅ Pass Train Dataset
    eval_dataset=eval_dataset,  # ✅ Pass Eval Dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ✅ Start Training
trainer.train()

<ipython-input-17-aeea7b6efdf8>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: harindu-20230712 (harindu-20230712-informatics-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.389400,3.183998
2,2.342400,2.047691
3,1.309200,1.125377
4,0.961000,0.835561
5,0.755500,0.652289
6,0.708800,0.542857
7,0.533000,0.475944


TrainOutput(global_step=280, training_loss=1.6029437593051366, metrics={'train_runtime': 12857.373, 'train_samples_per_second': 0.087, 'train_steps_per_second': 0.022, 'total_flos': 292647075840000.0, 'train_loss': 1.6029437593051366, 'epoch': 7.0})

In [18]:
# ✅ Compute Perplexity (PPL)
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"🧮 Model Perplexity : {perplexity:.2f}")

🧮 Model Perplexity : 1.61


In [19]:
# ✅ Save the Fine-Tuned Model
model.save_pretrained("/content/drive/MyDrive/DSGP/models/gpt2_finetuned2")
tokenizer.save_pretrained("/content/drive/MyDrive/DSGP/models/gpt2_finetuned2")

print("✅ GPT-2 Fine-Tuning Completed! Model Saved in Google Drive!")

✅ GPT-2 Fine-Tuning Completed! Model Saved in Google Drive!


In [20]:
# ✅ Load fine-tuned GPT-2 model
gpt2_pipe = pipeline("text-generation", model="/content/drive/MyDrive/DSGP/models/gpt2_finetuned2")

Device set to use cpu


In [ ]:
while True:
    user_query = input("\n🔍 Enter a bird question (or type 'exit' to quit): ")
    if user_query.lower() == "exit":
        break

    retrieved_chunk = "Large head, electric-blue back and wings, heavy red bill, snow-white throat patch."  # Use FAISS retrieval instead

    prompt = f"Instruction: Enhance the following bird description with a smooth and engaging narrative.\nQuestion: {user_query}\nRetrieved Chunk: {retrieved_chunk}\nEnhanced Response:"

    output = gpt2_pipe(
        prompt,
        max_new_tokens=50,
        temperature=0.5,
        top_p=0.9,
        repetition_penalty=1.4,
        return_full_text=False  # ✅ Fix: Only return generated response
    )

    print(f"\n🦜 GPT-2 Response: {output[0]['generated_text']}")



🦜 GPT-2 Response:  The range of this species includes : Medium to large in sizes; has been extensively studied by conservationists around Asia for its ability as well ornaments such on clothing worn while swimming (elevated neck feathers). It also exhibits distinctive black streaks across
